## Jupyter Notebook for ner_inf component

### Critical Instructions

1. Cell 1 contains component run parameters - change these as required when testing. These parameters will be passed automatically to the notebook when it is run on the xpresso server

2. Cell 2 contains the component class code with the following methods. You may optionally overwrite these, as required;
      - Constructor - you can overwrite the constructor if required (optional)
      - on_pause - this is called by xpresso when the user pauses an experiment - use it to save your component state, and perform any cleanup required
      - on_restart - this is called by xpresso when the user restarts an experiment - use it to reload your component state
      - on_terminate - this is called by xpresso when the user terminates an experiment - use it to perform any cleanup required before stopping the notebook
      - on_complete - this is called by xpresso when the proxy.completed method is called within the notebook. Use this to perform any completion operations (e.g., save the model to proxy.OUTPUT_DIR, any cleanup actions, etc.)

### Execution Instructions
1. Set the run parameters in Cell 1
2. Implement any component methods you want in Cell 2 (optional)
3. Run the first two cells
4. Your component code should appear in subsequent cells;
      - Use proxy.logger to log messages (see example provided)
      - Use proxy.report_status to report status back to xpresso
      - Use proxy.report_kpi_metrics to report KPIs back to xpresso
      - Use proxy.report_timeseries_metrics to report timeseries data back to xpresso
      - Use proxy.completed to indicate to xpresso that execution is complete (this will trigger a call to your component's on_complete method)

#### Please reach out to xpresso.ai team for any issues or concerns

In [ ]:
# Variables in this cell are tagged as "parameters" &
# will be replaced as command-line arguments during deployment

# === Default command-line arguments ===
enable_local_execution = True
xpresso_run_name = "Local Execution"
params_filename = None
params_commit_id = None
training_branch = None
training_dataset = None
training_data_version = None
key_metric = None

In [ ]:
"""
    Generated python code for component. We define the proxy code that will be used for execution in this cell.
    
    Component flow is as follows
    A. Local Execution - User is supposed to execute the NB on JupyterHub server for testing purposes.
        1. The first 2 cells of the notebook should be run - these create the component object and a proxy object
        2. Other cells can contain component code 
        3. Developer may make calls to proxy.report_status, proxy.report_kpi_metrics and 
            proxy.report_timeseries_metrics. These are mapped to local proxy methods which print information
            on the console
        4. Developer may make calls to proxy.completed method - this calls the on_complete method of the component
            - Developer can implement this method to perform any cleanup tasks


    B. Remote Execution - xpresso Controller calls xprbuild/system/linux/run.sh, which runs this notebook
        1 The first 2 cells of the notebook should be run - these create the component object and a proxy object
        2. Other cells can contain component code 
        3. Developer may make calls to proxy.report_status, proxy.report_kpi_metrics and 
            proxy.report_timeseries_metrics. These are mapped to remote proxy methods which display information
            on the kubeflow console or xpresso UI as required
        4. Developer may make calls to proxy.completed method - this calls the on_complete method of the component
            - Developer can implement this method to perform any cleanup tasks
        5. Developer may implement on_pause, on_restart and on_terminate to handle interrupts
"""


__author__ = ["### Author ###"]


class NerInf():
    def __init__(self):
        """
        Constructor
            Note: Constructor cannot take any parameters
            Initialize all the required constants and data here
        """
        pass
    
    def on_pause(self, push_exp=False):
        """
        This method is called by the local/remote proxy pause method
        In remote execution, this would be called when a user pauses the pipeline from the UI
        In local execution, this would be called from a separate thread for testing pause
        Developers must implement any cleanup required in this method,
        and save state of the program to disk
        Args:
            push_exp: Whether to push the data present in the output folder
               to the versioning system. This is required once training is
               completed and model needs to be versioned
        """
        # === Your pause code goes here ===
        pass

    def on_restart(self):
        """
        This method is called by the local/remote proxy restart method
        In remote execution, this would be called when a user restarts the pipeline from the UI
        In local execution, this would be called from a separate thread for testing restart
        Developers should implement the logic to
        reload the state of the previous run.
        """
        # === Your restart code goes here ===
        pass

    
    def on_terminate(self):
        """
        This method is called by the proxy terminate method
        In remote execution, this would be called when a user terminates the pipeline from the UI
        In local execution, this would be called from a separate thread for testing termination
        Developers must implement any cleanup required in this method
        """
        # === Your termination code goes here ===
        pass
    
    def on_complete(self, push_exp=False, success=True):
        """
        This method is called by the local/remote proxy completed method
        Developers must implement any cleanup required in this method

        Args:
            push_exp: Whether to push the data present in the output folder
               to the versioning system. This is required once training is
               completed and model needs to be versioned
            success: Use to handle failure cases
        """
        # === Your completion code goes here ===
        pass


component_object = NerInf()

# === Initialize Proxy ===
if enable_local_execution:
    from local_proxy import LocalProxy
    proxy = LocalProxy(
        component_object, xpresso_run_name, params_filename, params_commit_id)
else:
    from app.remote_proxy import RemoteProxy
    proxy = RemoteProxy(
        component_object, xpresso_run_name, params_filename, params_commit_id, 
        training_branch, training_dataset, training_data_version, key_metric)


In [ ]:
import logging
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('all')

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
import re
import pickle
from os.path import exists


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from polyfuzz import PolyFuzz
from polyfuzz.models import Embeddings
from flair.embeddings import TransformerWordEmbeddings

In [ ]:
base_path = "/project/pipelines/ner-topic/intermed_files/"

model_full_path = base_path + "ner_topic_model.h5"
model = load_model(model_full_path)


with open(base_path + 'tokenizer.pkl', 'rb') as fp_:
    tokenizer = pickle.load(fp_)

base_dic_df = pd.read_csv(base_path + 'entity_dic_filtered.csv')

base_dic = {}
for i in base_dic_df.index:
    base_dic[base_dic_df['lemma_word_phrase_updated'][i]] = base_dic_df['derivedCat'][i]
base_dic = base_dic

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

embeddings = TransformerWordEmbeddings('bert-base-multilingual-cased')
bert = Embeddings(embeddings, min_similarity=0, model_id="BERT")
models_ = PolyFuzz(bert)

In [ ]:
def clean_message(chat):
    chat['cleaned_message'] = chat['message'].apply(lambda x: " ".join(x.lower().split()))
    chat['tokenized'] = ''
    for i in chat.index:
        words_lis =  chat['cleaned_message'][i].split()
        words_lis = [re.sub("[^A-Za-z0-9]+", " ", j) for j in words_lis]
        words_lis = [re.sub("[0-9]+", " number", j) for j in words_lis]
        lem_words = [lemmatizer.lemmatize(j).lower() if len(j)>3 else j.lower() for j in words_lis]
        chat['tokenized'][i] = " ".join(lem_words).split()
    return chat

In [ ]:
def get_topics(chat,predictions,poly_dic):
    chat['derived_topics'] = ''
    chat['derived_topics_cat']=''
    p = 0
    fuz_list={}
    poly_list = {}
    for  i in chat.index:
        out = predictions[p]
        words_ = []
        for j in range(0,len(out)):
            if out[j]!='O':
                if out[j]== 'B-entity':
                    words_.append(chat['tokenized'][i][j])
                elif out[j]== 'I-entity':
                    if len(words_)>0:
                        words_[len(words_)-1] = words_[len(words_)-1]+' '+ chat['tokenized'][i][j]
                    else:
                        words_.append(chat['tokenized'][i][j])
        chat['derived_topics'][i] = words_
        cat_ = []
        for k in words_:
            if k in base_dic.keys():
                cat_.append(base_dic[k])
            elif k in poly_dic.keys():
                cat_.append(poly_dic[k])
            else:
                fuz_match = process.extractOne(k,base_dic.keys())
                #if fuz_match[1]>85:
                fuz_list[k]=fuz_match[0]
                cat_.append(base_dic[fuz_match[0]])
                #else:
                #    poly_list[k]='tbd'
                #    cat_.append('tbd')
        chat['derived_topics_cat'][i] = cat_
        p=p+1   
    return chat

In [ ]:
input_path = "/project/pipelines/ner-inf-top/input_file/"
chat = pd.read_csv(input_path + "input.csv")
chat = clean_message(chat)
chat['tokenized_joined'] = chat['tokenized'].apply(lambda x: " ".join(x))
Max_Sequence_Length = 30
seq = tokenizer.texts_to_sequences(chat['tokenized_joined'])
data = pad_sequences(seq, maxlen = Max_Sequence_Length,padding='post',truncating = 'post')

In [ ]:
pred = model.predict(data,batch_size = 128)
ids_to_labels= {0: 'O',1: 'B-entity', 2: 'I-entity'}
labels_to_ids = {'B-entity': 1, 'O': 0, 'I-entity': 2}
predictions = []
for i in range(0,len(pred)):
    inner = ['O']*len(pred[i])
    for j in range(0,len(inner)):
        if data[i][j]==0:
            break
        inner[j] = ids_to_labels[np.argmax(pred[i][j])]
    predictions.append(inner)
keys = []
for i in range(0,len(predictions)):
    for j in range(0,len(predictions[i])):
        if predictions[i][j] != 'O':
            if predictions[i][j] == 'B-entity':
                keys.append(chat['tokenized'][i][j])
            elif predictions[i][j] == 'I-entity':
                keys[len(keys)-1] = keys[len(keys)-1] + " "+ chat['tokenized'][i][j]

fp = list(set(keys) - set(base_dic.keys()))
poly_dic = {}   
poly_df = pd.DataFrame()

if len(fp)>0:
    from_list = list(set(fp))
    to_list = list(base_dic.keys())
    models_.match(from_list, to_list)
    poly_df = models_.get_matches()
    poly_df['to_cate'] = poly_df['To'].apply(lambda x: base_dic[x])
    for i in poly_df.index:
        poly_dic[poly_df['From'][i]]= poly_df['to_cate'][i]

chat = get_topics(chat,predictions,poly_dic)


In [ ]:
output_path = "/project/pipelines/ner-inf-top/output_file/"
chat.to_csv(output_path + 'output_topic.csv',index=False)
file_exists = exists(output_path + 'new_topic.csv')
if file_exists==True:
    poly_df_old = pd.read_csv(output_path + 'new_topic.csv')
    if len(poly_df)>0:
        poly_df_new = poly_df_old.append(poly_df).drop_duplicates(subset =['From'])
        poly_df_new.to_csv(output_path + 'new_topic.csv',index=False)
else:
    if len(poly_df)>0:
        poly_df.to_csv(output_path + 'new_topic.csv',index=False)

In [ ]:
# sample code to access run parameters
#print(proxy.run_parameters)

In [ ]:
# sample code for logger
#proxy.logger.info("Hello World")

In [ ]:
# sample code to report KPI metrics
#proxy.report_kpi_metrics({"key": 123})

In [ ]:
# sample code to report Time-series metrics
#proxy.report_timeseries_metrics({"key": 1})

In [ ]:
# sample code to call completed method
proxy.completed(push_exp=True, success=True)